In [2]:
import pandas as pd
import numpy as np
import dmc
from process import processed_data

data = processed_data()
train_ids, test_ids = dmc.loading.load_ids('rawMirrored')
train, test = dmc.preprocessing.split_train_test(data, train_ids, test_ids)

# allocate memory
data = train_ids = test_ids = None
####
ensemble = dmc.ensemble.Ensemble(train, test)

splits = len(ensemble.splits)
scalers = [dmc.transformation.scale_features] * splits
ignore = ([None] * 3 + 
          [['returnQuantity', 'orderID', 'orderDate']] * 3 + 
          [None] * 2 + 
          [['returnQuantity', 'orderID', 'orderDate']] * 2)
ensemble.transform(scalers=scalers, ignore_features=ignore)

for k in ensemble.splits:
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.NaiveBayes
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    print(k, prec, len(ye))

PicklingError: Can't pickle <function Ensemble._transform_split at 0x7f2d10102268>: attribute lookup _transform_split on dmc.ensemble failed